# Experimento: amplitud en función de la distancia

Las ondas sísmicas son ondas mecánicas. A pesar de moverse en un medio muy complejo, exhibiendo comportamientos no lineales, podemos afirmar que siguen ciertas tendencias. Una de esas es el decaimiento de la amplitud en función de la distancia. 

Para ilustrarlo, vamos a tomar las componentes verticales de todos los sismómetros al momento del sismo de Jacó. Vamos a resumir su amplitud en un único valor y lo vamos a graficar contra la distancia. 

In [ ]:
# Importe los módulos necesarios: obspy, numpy, matplotlib
import obspy
import numpy as np
import matplotlib.pyplot as plt

# Inventario con la respuesta del instrumento y la posición
inv = obspy.read_inventory('datos/tc_stations.xml') + obspy.read_inventory('datos/ov_stations.xml')

def pre_process(file, inventory):
    # Cargue el Stream 
    stream = obspy.read(file)

    # Pre-process
    stream.merge(fill_value=0)
    stream.detrend(type='linear')
    stream.remove_response(inventory, output='DISP',  taper_fraction=0.1, water_level=0)
    stream.filter('lowpass', freq=0.2)
    return stream

jaco = pre_process('datos/sismo_jaco.mseed', inv) + pre_process('datos/ov-sismo_jaco.mseed', inv)
print(jaco.__str__(extended=True))

Ahora vamos a intentar resumir la amplitud de la onda en un único valor. Para ello vamos a tomar su valor absoluto y lo vamos a sumar por la duración de la señal.

El epicentro del sismo se localizó a 9.53° de latitud y -84.51° de longitud. Utilice el método `get_coordinates()` del objeto `inventary` para obtener la posición de cada estación. Calcule la distancia euclideana entre el epicentro y la posición de cada una de las estaciones.

In [ ]:
amplitud  = []
distancia = []


# utilice un for-loop para recorrer todas las estaciones. 
for estacion in jaco:
    nombre = '.'.join([estacion.stats[i] for i in ['network', 'station', 'location', 'channel']])
    
    # tome el valor absoluto de la señal y súmela
    amplitud.append(np.sum(abs(estacion.data)))
    
    # Calcule la distancia al hipocentro
    delta_lat  = inv.get_coordinates(nombre)['latitude'] - 9.53
    delta_long = inv.get_coordinates(nombre)['longitude'] - (-84.51)
    distancia.append(np.hypot(delta_lat, delta_long))
    
outliers = amplitud < np.mean(amplitud) + 4 * np.std(amplitud)

amplitud  = [amplitud[i] for i in range(len(outliers)) if outliers[i]]
distancia = [distancia[i] for i in range(len(outliers)) if outliers[i]]
    
print(amplitud)
print(distancia)

Ahora, grafique la amplitud en función de la distancia

In [ ]:
plt.scatter(distancia, amplitud)
plt.xlabel('Distancia (grados decimales)')
plt.ylabel('Desplazamiento total (m)')
plt.show()

Calculemos una línea de tendencia. 

In [ ]:
tendencia = np.polyfit(distancia, amplitud, 1)
print(tendencia)

In [ ]:
t = np.poly1d(tendencia)
x = np.linspace(0,1.75,40)
t_y = np.array([t(y) for y in x])

plt.scatter(distancia, amplitud)
plt.scatter(x, t_y)
plt.xlabel('Distancia (grados decimales)')
plt.ylabel('Desplazamiento total (m)')
plt.show()